In [ ]:
from discretize import TensorMesh
from SimPEG import maps, data
import numpy as np
from matplotlib.colors import LogNorm
from simpegEM1D import (
    GlobalEM1DSimulationTD, GlobalEM1DSurveyTD, get_vertical_discretization_time, EM1DSurveyTD
)
from pymatsolver import PardisoSolver
%pylab inline

# Seting up a 2D mesh and model

In [ ]:
from simpegEM1D import skytem_HM_2015
wave = skytem_HM_2015()
time = np.logspace(-5, -2, 21)
# hz = get_vertical_discretization_time(time, facter_tmax=0.5, factor_tmin=10.)
hz = 1*np.arange(30)**1.1
time_input_currents = wave.current_times[-7:]
input_currents = wave.currents[-7:]
from scipy.spatial import Delaunay
def PolygonInd(mesh, pts):
    hull = Delaunay(pts)
    inds = hull.find_simplex(mesh.gridCC)>=0
    return inds
n_sounding = 10
dx = 100.
hx = np.ones(n_sounding) * dx
mesh = TensorMesh([hx, hz], x0='00')
inds =  mesh.gridCC[:,1]<25
sigma = np.ones(mesh.nC) * 1./20.
sigma[inds] = 1./20.
x0 = np.r_[0., 75.]
x1 = np.r_[dx*n_sounding, 75.]
x2 = np.r_[dx*n_sounding, 90.]
x3 = np.r_[0., 200.]
pts = np.vstack((x0, x1, x2, x3, x0))
poly_inds = PolygonInd(mesh, pts)
sigma[poly_inds] = 1./50.
sigma_em1d = sigma.reshape(mesh.vnC, order='F').flatten()
cb = plt.colorbar(
    mesh.plotImage(1./sigma, grid=False, clim=(10, 50),pcolorOpts={"norm":LogNorm()})[0],
    fraction=0.03, pad=0.04
)

plt.ylim(mesh.vectorNy.max(), mesh.vectorNy.min())
plt.gca().set_aspect(1)

In [ ]:
from SimPEG import (
    regularization, directives, inversion, inverse_problem, optimization, data_misfit, utils
)
from simpegEM1D import get_2d_mesh, LateralConstraint

In [ ]:
x = mesh.vectorCCx
y = np.zeros_like(x)
z = np.ones_like(x) * 30.
rx_locations = np.c_[x, y, z]
src_locations = np.c_[x, y, z]
topo = np.c_[x, y, z-30.].astype(float)

mapping = maps.ExpMap(mesh)

survey = GlobalEM1DSurveyTD(
    rx_locations = rx_locations,
    src_locations = src_locations,    
    topo = topo,
    time = [time for i in range(n_sounding)],
    src_type = np.array(["CircularLoop"], dtype=str).repeat(n_sounding),
    rx_type = np.array(["dBzdt"], dtype=str).repeat(n_sounding),    
    wave_type = np.array(["stepoff"], dtype=str).repeat(n_sounding),    
    field_type = np.array(["secondary"], dtype=str).repeat(n_sounding),    
    a = np.array([13.]).repeat(n_sounding),
    input_currents=[input_currents for i in range(n_sounding)],
    time_input_currents=[time_input_currents for i in range(n_sounding)],
    base_frequency = np.array([20.]).repeat(n_sounding),
)

sim = GlobalEM1DSimulationTD(
    mesh, sigmaMap=mapping, survey=survey, hz=hz, parallel=False, n_cpu=2,
    Solver=PardisoSolver
)

m = np.log(sigma_em1d)
d_true = sim.dpred(m)

In [ ]:
DOBS = d_true.reshape(( sim.n_sounding, time.size))
for i_time in range(time.size):
    plt.semilogy(mesh.vectorCCx, -DOBS[:,i_time], 'k')

In [ ]:
mesh_reg = get_2d_mesh(n_sounding, hz)
regmap = maps.IdentityMap(mesh_reg)
# mapping is required ... for IRLS
reg = LateralConstraint(
    mesh_reg, mapping=regmap,
    alpha_s = 1.,
    alpha_x = 1.,
    alpha_y = 1,    
)
xy = utils.ndgrid(np.arange(n_sounding), np.r_[0.])
reg.get_grad_horizontal(xy, hz, dim=2)

In [ ]:
std = 0.03
floor = 0.
std = 0.03
floor = 0.
np.random.seed(1)
uncert = std*abs(d_true)+floor
noise = std*np.abs(d_true)*np.random.randn(len(d_true))
d_obs = d_true + noise

dataObj = data.Data(survey, dobs=d_obs, noise_floor=uncert)

m0 = np.ones(mesh.nC) * np.log(1./20.)
dmisfit = data_misfit.L2DataMisfit(dataObj, sim)
dmisfit.W = 1./uncert
# p = 0
# qx, qz = 1., 1.
# reg.norms = np.c_[p, qx, qz, 0.]
# IRLS = Directives.Update_IRLS(
#     maxIRLSiter=5, minGNiter=1, fix_Jmatrix=True,
#     betaSearch=False
# )
opt = optimization.InexactGaussNewton(maxIter = 30)
invProb = inverse_problem.BaseInvProblem(dmisfit, reg, opt)
beta = directives.BetaSchedule(coolingFactor=2, coolingRate=1)
betaest = directives.BetaEstimate_ByEig(beta0_ratio=1.)
target = directives.TargetMisfit()
inv = inversion.BaseInversion(invProb, directiveList=[beta,betaest,target])
# inv = Inversion.BaseInversion(invProb, directiveList=[IRLS,betaest])
# prob.counter = opt.counter = Utils.Counter()
opt.LSshorten = 0.5
opt.remember('xc')
mopt = inv.run(m0)

In [ ]:
sigma_est = utils.mkvc((np.exp(mopt)).reshape((sim.n_sounding, sim.n_layer)))
# sigma_est_l2 = Utils.mkvc((np.exp(invProb.l2model)).reshape((prob.n_sounding, prob.n_layer)))

In [ ]:
import matplotlib
matplotlib.rcParams['font.size'] = 14

In [ ]:
cb = plt.colorbar(
    mesh.plotImage(
        1./sigma, grid=True, clim=(10, 50),pcolorOpts={"norm":LogNorm()},
        gridOpts={'alpha':0.2, 'color':'grey'}
    )[0],
    fraction=0.02, pad=0.04
)

plt.ylim(mesh.vectorNy.max(), mesh.vectorNy.min())
plt.gca().set_aspect(1)

plt.title("True model")


cb = plt.colorbar(
    mesh.plotImage(
        1./sigma_est, grid=True, clim=(10, 50), pcolorOpts={"norm":LogNorm()},
        gridOpts={'alpha':0.2, 'color':'grey'}
    )[0],
    fraction=0.02, pad=0.04
)
plt.ylim(mesh.vectorNy.max(), mesh.vectorNy.min())
plt.gca().set_aspect(1)
plt.title("Laterally-contrained model: L0 L2")

In [ ]:
# survey.n_time

In [ ]:
n_time = time.size
PRED = invProb.dpred.reshape((sim.n_sounding, n_time))

In [ ]:
for i_time in range(n_time):
    plt.semilogy(mesh.vectorCCx, -DOBS[:,i_time], 'k')
    plt.semilogy(mesh.vectorCCx, -PRED[:,i_time], 'kx', ms=1)

In [ ]:
J_sigma = sim.getJ_sigma(m)
J = J_sigma

In [ ]:
from pymatsolver import Pardiso

In [ ]:
reg = LateralConstraint(
    mesh_reg, mapping=regmap,
    alpha_s=0.,
    alpha_x = 1.,
    alpha_y = 1.,    
)
xy = utils.ndgrid(np.arange(n_sounding), np.r_[0.])
reg.get_grad_horizontal(xy, hz, dim=2)

In [ ]:
betaest.initialize()

In [ ]:
betaest.beta0 

In [ ]:
n_sample = 100
Wd = dmisfit.W
beta_post = betaest.beta0 * 0.01
Hd = J.T.dot((Wd.T*(Wd*J)))
Hm = reg.W.T*reg.W
H = Hd + beta_post * Hm
# L = np.linalg.cholesky(H)
Hinv = Pardiso(H, is_symmetric=True, is_positive_definite=True)
Hinv.factor()
rhs = np.random.randn(H.shape[0], n_sample)
x = Hinv.solver.run_pardiso(333, rhs=rhs)

In [ ]:
x_tmp = np.random.rand(mopt.size)
Wd = dmisfit.W
beta_0 = np.linalg.norm(Wd*(J*x_tmp))**2 / np.linalg.norm((reg.W*x_tmp))**2

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)
ii = 2
sigma_est = utils.mkvc((np.exp(mopt)).reshape((sim.n_sounding, sim.n_layer)))
cb = plt.colorbar(
    mesh.plotImage(
        1./sigma_est, grid=True, clim=(20, 50),pcolorOpts={"norm":LogNorm()},
        gridOpts={'alpha':0.2, 'color':'grey'}, ax=ax1
    )[0],
    fraction=0.02, pad=0.04, ax=ax1
)
sigma_est = utils.mkvc((np.exp(mopt+x[:,ii])).reshape((sim.n_sounding, sim.n_layer)))
cb = plt.colorbar(
    mesh.plotImage(
        1./sigma_est, grid=True, clim=(20, 50),pcolorOpts={"norm":LogNorm()},
        gridOpts={'alpha':0.2, 'color':'grey'}, ax=ax2
    )[0],
    fraction=0.02, pad=0.04, ax=ax2
)
for ax in [ax1, ax2]:
    ax.set_ylim(mesh.vectorNy.max(), mesh.vectorNy.min())
    ax.set_aspect(1)
plt.tight_layout()

In [ ]:
from simpegEM1D import set_mesh_1d, plotLayer
mesh_1d = set_mesh_1d(hz)

In [ ]:
ii=1
n_samples = [3, 5, 20, 50, 80, 100, 200, 300, 400, 500, 600, 700, 1000, 10000]
lows = []
highs = []
perc_low = 2.1
for n_sample in n_samples:
    rhs = np.random.randn(H.shape[0], n_sample)
    x = Hinv.solver.run_pardiso(333, rhs=rhs)
    ind_start = ii*hz.size
    ind_end = ii*hz.size + hz.size
    tmp = mopt[ind_start:ind_end, np.newaxis] + x[ind_start:ind_end,:]
    low = np.percentile(tmp, perc_low, axis=1)
    high = np.percentile(tmp, 100-perc_low, axis=1)
    lows.append(low)
    highs.append(high)
# for jj in range(len(n_samples)):
jj = 12
print (n_samples[jj])
plotLayer(1./np.exp(lows[jj]), mesh_1d,  **{'color':'b', 'lw':1})
plotLayer(1./np.exp(highs[jj]), mesh_1d, **{'color':'b', 'lw':1})

plotLayer(1./np.exp(lows[len(n_samples)-1]), mesh_1d,  **{'color':'g', 'lw':1})
plotLayer(1./np.exp(highs[len(n_samples)-1]), mesh_1d, **{'color':'g', 'lw':1})
plotLayer(
    1./np.exp(mopt[ind_start:ind_end, np.newaxis]), 
    mesh_1d, showlayers=False,**{'color':'r', 'lw':2}
)
plt.xlim(2, 200)
# 500 samples    

In [ ]:
ii = 12
dpred = sim.dpred(mopt+x[:,ii])
n_time = time.size
PRED = dpred.reshape((sim.n_sounding, n_time))

for i_time in range(n_time):
    plt.semilogy(mesh.vectorCCx, -DOBS[:,i_time], 'k')
    plt.semilogy(mesh.vectorCCx, -PRED[:,i_time], 'k--', ms=5)